In [ ]:
import pyorc
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
radar = pd.read_csv('july1-test1.csv')
print(radar.columns)
radar

In [ ]:
print(radar['Time stamp'].dtype)
radar['Time stamp']

In [ ]:
radar['Time stamp'] = pd.to_datetime(radar['Time stamp'], format="%m/%d/%Y %I:%M")
print(radar['Time stamp'].dtype)
radar['Time stamp']

In [ ]:
fig, ax = plt.subplots(2, sharex=True)

sns.lineplot(
    data=radar,
    x='Time stamp',
    y=' Average velocity value',
    ax = ax[0]
)
sns.lineplot(
    data=radar,
    x='Time stamp',
    y='Tilt angle value',
    ax=ax[1]
)

In [ ]:
ds = xr.open_dataset('video_capture_2024-07-01_12-22-37_velocimetry_results.nc')
ds = ds.mean(dim="time", keep_attrs=True)

df = ds.to_dataframe().reset_index(); ds.close()
df['velocity'] = np.sqrt(df.v_x**2 + df.v_y**2)
df

In [ ]:
ds = xr.open_dataset('video_capture_2024-07-01_12-22-37_velocimetry_results.nc')

ds.velocimetry.mask.corr(inplace=True)
ds.velocimetry.mask.minmax(inplace=True)
ds.velocimetry.mask.rolling(inplace=True)
ds.velocimetry.mask.outliers(inplace=True)
ds.velocimetry.mask.variance(inplace=True)
ds.velocimetry.mask.angle(angle_tolerance=0.5*np.pi)
ds.velocimetry.mask.count(inplace=True)
ds.velocimetry.mask.window_mean(wdw=2, inplace=True, tolerance=0.5, reduce_time=True)

ds = ds.mean(dim="time", keep_attrs=True)

df_post = ds.to_dataframe().reset_index(); ds.close()
df_post['velocity'] = np.sqrt(df_post.v_x**2 + df_post.v_y**2)
df_post

In [ ]:
print('MEAN VELOCITY')
print(f"{'Radar':<24}: {np.mean(radar[' Average velocity value'])}")
print(f"{'PyORC':<24}: {np.mean(df['velocity'])}")
print(f"{'PyORC w/ post-processing':<24}: {np.mean(df_post['velocity'])}")

range = [0,1.5]
n=64

plt.figure(figsize=(7, 5))

sns.histplot(
    data=radar,
    x=' Average velocity value',
    binrange=range,
    bins=n,
    label='radar'
)
sns.histplot(
    data=df,
    x='velocity',
    binrange=range,
    bins=n,
    label='PyORC'
)
sns.histplot(
    data=df_post,
    x='velocity',
    binrange=range,
    bins=n,
    label='PyORC w/ post-processing'
)

plt.xlabel('velocity')
plt.legend()